# Analysis procedure for snmulti-QTL

## Get reference data (Optional)
Download the reference data needed. In fact this step is not necessary, the reference data can be downloaded by ` synapse get -r syn36416587 `

In [ ]:
nohup sos run pipeline/reference_data.ipynb download_hg_reference --cwd ../input/reference_data &
nohup sos run pipeline/reference_data.ipynb download_gene_annotation --cwd ../input/reference_data &
nohup sos run pipeline/reference_data.ipynb download_ercc_reference --cwd ../input/reference_data &
nohup sos run pipeline/reference_data.ipynb download_dbsnp --cwd ../input/reference_data &

In [19]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos run pipeline/reference_data.ipynb hg_reference \
    --cwd ../input/reference_data \
    --ercc-reference ../input/reference_data/ERCC92.fa \
    --hg-reference ../input/reference_data/GRCh38_full_analysis_set_plus_decoy_hla.fa \
    --container ../xqtl-pipeline-container/rna_quantification.sif    &

[1] 47287


: 1

In [22]:
nohup sos run pipeline/reference_data.ipynb hg_gtf \
    --cwd ../input/reference_data \
    --hg-gtf ../input/reference_data/Homo_sapiens.GRCh38.103.chr.gtf \
    --hg-reference ../input/reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy.fasta \
    --container ../xqtl-pipeline-container/rna_quantification.sif   --stranded -s build  &

[1] 47352


: 1

## Archived: Gene exp matrix generation
This is to process the count matrix. This step is replaced by the SE_qc step via dreamlet below

In [ ]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos run pipeline/pseudobulk_expression_QC_and_normalization.ipynb \
    --phenoFile  ~/snmulti_QTL/input/gene_exp/*.count_matrix \
    --cwd ~/snmulti_QTL/output/molecular_phenotype/gene_exp/ \
    --BrainRegionList  /hpc/users/sunh14/snmulti_QTL/input/BrainRegionList" \
    -J 40 -q premium -c ~/minerva.yml  -s force &

In [ ]:
nohup sos run pipeline/pseudobulk_expression_QC_and_normalization.ipynb \
    --phenoFile  ~/snmulti_QTL/input/gene_exp/*.count_matrix \
    --cwd ~/snmulti_QTL/output/molecular_phenotype/gene_exp/ \
    --BrainRegionList  /hpc/users/sunh14/snmulti_QTL/input/BrainRegionList" \
    -J 40 -q premium -c ~/minerva.yml  -s force &

In [25]:
sos run pipeline/pseudobulk_expression_QC_and_normalization.ipynb \
    --phenoFile  ~/snmulti_QTL/input/gene_exp/*.count_matrix \
    --cwd ~/snmulti_QTL/output/molecular_phenotype/gene_exp/ \
    --BrainRegionList  /hpc/users/sunh14/snmulti_QTL/input/BrainRegionList \
    -J 40 -q premium -c ~/minerva.yml  -s force &

[1] 82193


## Gene exp matrix generation via dreamlet

From get the pseudobulk count matrix from the `SingleCellExperiment` R data object and extract the count table for each brain region and celltypes

In [230]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos run pipeline/pseudobulk_expression_QC_and_normalization.ipynb SE_qc \
    --phenoFile  /sc/arion/projects/CommonMind/roussp01a/snmulti/step3/files/pb4dreamlet_wnn_celltype.Rdata \
    --cwd ~/snmulti_QTL/output/molecular_phenotype/gene_exp_dreamlet/ \
    --celltypes ../Analysis_notebook/Celltypes.tsv --container container/panos.sif --entrypoint "" \
    --BrainRegionList  ../Analysis_notebook/BrainRegion  -s force &

[3] 90174


: 1

In [1]:
head ../Analysis_notebook/BrainRegion

region
EC
PFC
STG


In [2]:
head ../Analysis_notebook/Celltypes.tsv

Cellltypes
Astro
Endo
EN_L2_3|3_5_IT_mixture
EN_L2_3_IT_1
EN_L2_3_IT_2
EN_L3_5_IT_1
EN_L3_5_IT_2
EN_L3_5_IT_3
EN_L3_5_IT_4


## ATAC-Seq matrix generation via dreamlet

In [335]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos run pipeline/pseudobulk_expression_QC_and_normalization.ipynb SE_qc \
    --phenoFile  /sc/arion/projects/CommonMind/roussp01a/snmulti/step3/files/ArchR/save3/pb4dreamlet_subclass_ATAC.Rdata \
    --cwd ~/snmulti_QTL/output/molecular_phenotype/ATAC_dreamlet/ \
    --celltypes ../Analysis_notebook/Celltypes.tsv --container container/panos.sif --entrypoint "" \
    --BrainRegionList  ../Analysis_notebook/BrainRegion  -s force  -J 90 -q premium -c ~/minerva.yml --mem 60G &  

[1] 2867


: 1

## Genotype 

### Extract samples from the two VCF
See the Genotype.ipynb. 8 samples are from ADSP and 240 samples from PsychAD. Extract them for merging

In [3]:
nohup bcftools view -S ../input/snmulti_QTL_sample_list  \
/sc/arion/projects/roussp01a/deepika/merging_psychAD_SNParray_WGS/common_variants_psychAD/psychAD_merged_common_variants_MSSM_CMC_HBCC_RUSH_ADSP_chr1to22.vcf.gz  \
-Oz -o ../input/psychAD_merged_common_variants_MSSM_CMC_HBCC_RUSH_ADSP_chr1to22_analysis.vcf.gz --force-samples &


[1] 83666


: 1

In [65]:
nohup bcftools view -S ../input/snmulti_QTL_sample_list_adsp  \
/sc/arion/projects/psychAD/genotypes/ADSP/vcfs/gcad.preview.compact_filtered.r3.wgs.16906.GATK.2020.05.26.biallelic.genotypes.ALL.vcf.gz \
-Oz -o ../input/gcad.preview.compact_filtered.r3.wgs.16906.GATK.2020.05.26.biallelic.genotypes.analysis.vcf.gz --force-samples &

[1] 74854


: 1

## VCF QC 

In [5]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos run  pipeline/VCF_QC.ipynb qc \
    --genoFile ../input/snmulti_QTL_analysis_psychAD_ADSP.vcf \
    --dbsnp-variants ../input//reference_data/00-All.add_chr.variants.gz  \
    --cwd ~/snmulti_QTL/output/Genotype/ \
    --reference_genome ../input/reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy.fasta \
    --container ../xqtl-pipeline-container/bioinfo.sif -J 1 -q premium -c ~/minerva.yml --mem 120G &

[1] 73379


: 1


The psych AD genotype only has 19141160 variants. Most of the snp in the input vcf came from the ADSP samples which has 0.1 billion snps but only 8 samples. That is why after variant level QC, the remaining snp is relatively small.
```
bcftools view -H psychAD_merged_common_variants_MSSM_CMC_HBCC_RUSH_ADSP_chr1to22_analysis.vcf.gz | wc -l &
(py3.11) [sunh14@lc03e08 input]$ 19141160
```

In [89]:
cat  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.vcf.stdout

output_info: /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.vcf.gz
output_size: 5.3G
output_rows: 15881136
output_column: 257
output_header_row: 3510
output_preview:
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	G-MSBB-MB000040-BR-MSBB-70659	207822
chr1	779825	chr1:779825:C:G	C	G	503500	PASS	ori_AN=33812;BaseQRankSum=0;DB;ori_DP=638078;ExcessHet=0.0002;FS=0;InbreedingCoeff=0.0339;MQ=58.95;MQRankSum=0;NEGATIVE_TRAIN_SITE;POSITIVE_TRAIN_SITE;QD=13.84;ReadPosRankSum=0.016;SOR=0.643;VQSLOD=0.185;culprit=MQ;MAF=0.0181452;R2=0.37856;IMPUTED;DP=1226211;AF=0.0181452;ori_AC=875;ori_AF=0.025;MLEAC=892;MLEAF=0.025;AN=496;AC=9;F_MISSING=0;VD=0;NS=248;AC_Het=5;AC_Hom=4;AC_Hemi=0;HWE=0.00152798;ExcHet=0.99999;RSID=rs141090730	GT:AD:DP:GQ:PL:DS:HDS:GP	0/0:.:.:.:.:.:.:.	0|0:.:.:.:.:0:0,0:1,0,0
chr1	779929	chr1:779929:AG:A	AG	A	83161.4	PASS	ori_AN=33812;BaseQRankSum=0;DB;ori_DP=633555;ExcessHet=4.0759;FS=0;InbreedingCoeff=-0.0037;MQ=59.22;MQRankSum=

### VCF to Plink

In [116]:
nohup sos run pipeline/genotype_formatting.ipynb vcf_to_plink \
    --genoFile ../output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.vcf.gz \
    --cwd ~/snmulti_QTL/output/Genotype/ \
    --container ../xqtl-pipeline-container/bioinfo.sif &

[2] 272135


: 1

In [300]:
cat  /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.log

PLINK v2.00a3.7LM 64-bit Intel (24 Oct 2022)
Options in effect:
  --bfile /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc
  --geno 0.1
  --hwe 1e-08
  --mac 5.0
  --make-bed
  --memory 14400.0
  --mind 0.1
  --new-id-max-allele-len 1000
  --out /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc
  --set-all-var-ids chr@:#_$r_$a
  --threads 20

Hostname: lc03e08
Working directory: /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
Start time: Wed Dec 13 13:38:25 2023

Random number seed: 1702492705
192949 MiB RAM detected; reserving 14400 MiB for main workspace.
Using up to 20 threads (change this with --threads).
248 samples (0 females, 0 males, 248 ambiguous; 248 founders) loaded from
/hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.fam.
15877625 variants loaded from
/hpc/users/sunh14/snmulti_QTL/output/Genotype/snm

In [120]:
head  /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.bim

chr1	chr1:779825:C:G	0	779825	G	C
chr1	chr1:779929:AG:A	0	779929	A	AG
chr1	chr1:782207:G:A	0	782207	A	G
chr1	chr1:787965:A:C	0	787965	C	A
chr1	chr1:793571:C:T	0	793571	T	C
chr1	chr1:800177:GT:G	0	800177	G	GT
chr1	chr1:801327:GTAAA:G	0	801327	G	GTAAA
chr1	chr1:805036:A:G	0	805036	G	A
chr1	chr1:805477:G:A	0	805477	A	G
chr1	chr1:817125:A:G	0	817125	G	A


## Plink QC
~Half of the genotype are removed, I am beginning to cnecern


In [121]:
nohup sos run pipeline/GWAS_QC.ipynb qc_no_prune \
   --cwd ~/snmulti_QTL/output/Genotype \
   --genoFile /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.bed \
   --geno-filter 0.1 \
   --mind-filter 0.1 \
   --hwe-filter 1e-08   \
   --mac-filter 5 \
   --container ../xqtl-pipeline-container/bioinfo.sif  &

[2] 273499


: 1

In [125]:
cat  /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.log

PLINK v2.00a3.7LM 64-bit Intel (24 Oct 2022)
Options in effect:
  --bfile /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc
  --geno 0.1
  --hwe 1e-08
  --mac 5.0
  --make-bed
  --memory 14400.0
  --mind 0.1
  --new-id-max-allele-len 1000
  --out /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc
  --set-all-var-ids chr@:#_$r_$a
  --threads 20

Hostname: lc03e08
Working directory: /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
Start time: Wed Dec 13 13:38:25 2023

Random number seed: 1702492705
192949 MiB RAM detected; reserving 14400 MiB for main workspace.
Using up to 20 threads (change this with --threads).
248 samples (0 females, 0 males, 248 ambiguous; 248 founders) loaded from
/hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.fam.
15877625 variants loaded from
/hpc/users/sunh14/snmulti_QTL/output/Genotype/snm

## Plink by chrom

In [145]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos run pipeline/genotype_formatting.ipynb genotype_by_chrom \
    --genoFile /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.bed \
    --cwd ../output/Genotype \
    --chrom `cut -f 1 /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.bim | uniq | sed "s/chr//g"` \
    --container ../xqtl-pipeline-container/bioinfo.sif  &

[3] 285777


## GRM generation

In [152]:
nohup sos run pipeline/GRM.ipynb grm \
    --cwd ../output/GRM \
    --genoFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.txt \
    --container ../xqtl-pipeline-container/bioinfo.sif  &

[3] 4453


: 1

In [3]:
cat /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.txt

#id	#path
7	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.7.bed
15	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.15.bed
10	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.10.bed
2	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.2.bed
4	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.4.bed
22	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.22.bed
9	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.9.bed
17	/

## Examn relateness for the samples
This is to demostrate the is no related samples in the study.

In [172]:
sos run /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline/pipeline/GWAS_QC.ipynb king \
   --cwd recipe_testing/data_preprocessing/Astro.PFC/genotype_data \
   --genoFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.bed \
   --name Astro.PFC \
   --container container/bioinfo.sif \
   --walltime 48h  -J 50 -c /hpc/users/sunh14/minerva.yml -q premium --no-maximize_unrelated

/sc/arion/work/sunh14/conda/envs/py3.11/lib/python3.11/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)
INFO: Running king_1: Inference of relationships in the sample to identify closely related individuals
orit_res: span[hosts=1] rusage[mem=16384.00]
init mod: 
INFO: t6a24a65af17d33e2 submitted to premium with job id 108321615
INFO: king_1 output:   /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.PFC/genotype_data/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.kin0
INFO: Running king_2: Select a list of unrelated individual with an attempt to maximize the unrelated individuals selec

: 1

In [173]:
cat /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.PFC/genotype_data/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.kin0

#FID1	IID1	FID2	IID2	NSNP	HETHET	IBS0	KINSHIP


In [189]:
cat /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.PFC/genotype_data/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.related_id

In [241]:
sos run /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline/pipeline/GWAS_QC.ipynb king \
   --cwd recipe_testing/data_preprocessing/Astro.PFC/genotype_data \
   --genoFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.bed \
   --name Astro.PFC \
   --container container/bioinfo.sif 
#   --walltime 48h  -J 50 -c /hpc/users/sunh14/minerva.yml -q premium --no-maximize_unrelated -s force 

INFO: Running king_1: Inference of relationships in the sample to identify closely related individuals
INFO: king_1 (index=0) is ignored due to saved signature
INFO: king_1 output:   /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.PFC/genotype_data/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.kin0
INFO: Running king_2: Select a list of unrelated individual with an attempt to maximize the unrelated individuals selected from the data
INFO: king_2 (index=0) is ignored due to saved signature
INFO: king_2 output:   /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.PFC/genotype_data/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.related_id
INFO: Running king_4: Split genotype data into related and unrelated samples, if related individuals are detected
INFO: No related individuals detected from /sc/arion/projects/CommonMind/roussp01a/snm

In [198]:
cat /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.PFC/genotype_data/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.related.stderr

Error: No samples remaining after main filters.


In [195]:
ls /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.PFC/genotype_data/

snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.kin0
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.kin0.stderr
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.kin0.stdout
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.log
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.related.bed.step_king.stderr
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.related.filtered.extracted.bed
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.related_id
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.related.log
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.unrelated.bim
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.unrelated.fam
snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.Astro.PFC.unrelated.filtered.prune.bed
snmulti_QTL_a

## Automated runs
Please see recip_generation.ipynb in the same folder for details on how to generate the recipe for running the automated commands. 
#### Now the MMQTL run for every analysis i.e. condcut a univariate analysis for each row in the recipe. I believe this is the most robust and rigorous way for next people who run it. ####

In [241]:
tail test_recipe_script

INFO: Step resid_factor_exp (index=0) is ignored with signature constructed
INFO: Step resid_factor_exp (index=3) is ignored with signature constructed
INFO: resid_factor_exp output:   recipe_testing/data_preprocessing/Astro.EC/resid_phenotype/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.gz recipe_testing/data_preprocessing/Astro.PFC/resid_phenotype/pb4dreamlet_subclass.Astro.PFC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.gz... (4 items in 4 groups)
INFO: Running phenotype_partition_by_chrom: 
INFO: phenotype_partition_by_chrom (index=1) is completed.
INFO: phenotype_partition_by_chrom (index=0) is completed.
INFO: phenotype_partition_by_chrom (index=3) is completed.
INFO: phenotype_partition_by_chrom (index=2) is completed.
INFO: phenotype_partition_by_chrom output:   recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2c

In [232]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos run pipeline/mmQTL_analysis_commands.ipynb phenotype_partition_by_chrom \
    --recipe   test_recipe \
    --annotation_gtf /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.gtf   \
    --sample_participant_lookup ../input//snmulti_QTL_sample_participant_lookup_re  \
    --cwd recipe_testing  \
    --annotation_fasta  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy_ERCC.fasta \
    --genotype_list /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.region_list.txt   \
    --grmFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/GRM/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.loco_grm_list.txt \
    --region_list  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.unique.region_list   \
    --n_brain_region 3 \
    --queue     premium  --yml  ~/minerva.yml --run  --exe_dir /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline -s build > test_recipe_script  &

[5] 54613


: 1

If you want to see the detail script that will be run, please use the following commands: change `run` to `dryrun`, removing the `-s build`

In [ ]:
cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working
nohup sos dryrun pipeline/mmQTL_analysis_commands.ipynb MMQTL \
    --recipe   test_recipe \
    --annotation_gtf /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.gtf   \
    --sample_participant_lookup ../input//snmulti_QTL_sample_participant_lookup_re  \
    --cwd recipe_testing  \
    --annotation_fasta  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy_ERCC.fasta \
    --genotype_list /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.region_list.txt   \
    --grmFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/GRM/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.loco_grm_list.txt \
    --region_list  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.unique.region_list   \
    --queue     premium  --yml  ~/minerva.yml --run  --exe_dir /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline > test_recipe_script  &

This is one of the command modified (using head to see only the first few genes) from the command generator to test the mmQTL.

This is the genotype list that were input into the analysis. Each row is a gene and the genotype file is partitioned into chromosome

In [490]:
head /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.region_list.txt

#id	#path
ENSG00000223972	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.1.bed
ENSG00000278267	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.1.bed
ENSG00000243485	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.1.bed
ENSG00000227232	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.1.bed
ENSG00000284332	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.1.bed
ENSG00000237613	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.1.bed
ENSG00000268020	/sc/arion/projects/CommonMind/roussp01a/snmulti_QT

## Astrocyte mmQTL
This is the actual command used to generate the 3 Astrocyte MMQTL analysis. There could be some issue with the container/panos.sif in this step which was ran on and off.

In [ ]:
nohup sos run /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline/pipeline/MMQTL.ipynb MMQTL \
--genoFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.region_list.txt \
--phenoFile recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.phenotype_by_chrom_files.region_list.txt\
recipe_testing/data_preprocessing/Astro.PFC/phenotype_data/pb4dreamlet_subclass.Astro.PFC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.phenotype_by_chrom_files.region_list.txt \
recipe_testing/data_preprocessing/Astro.STG/phenotype_data/pb4dreamlet_subclass.Astro.STG.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.phenotype_by_chrom_files.region_list.txt \
--cwd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/association_scan/MMQTL --grm_file /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/GRM/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.loco_grm_list.txt \
--region_list  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.unique.region_list   \
--container container/panos.sif  --name test_recipe -n > mmqtl_script &

In [233]:
rm recipe_testing/data_preprocessing/Astro*/phenotype_data/pb4dreamlet_subclass.A*.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.phenotype_by_chrom_files.region_list.txt

## 84 celltypes mmQTL

In [ ]:
nohup sos run pipeline/mmQTL_analysis_commands.ipynb MMQTL \
    --recipe   /hpc/users/sunh14/snmulti_QTL/input/snmulti_QTL_analysis_recipe.tsv \
    --annotation_gtf /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.gtf   \
    --sample_participant_lookup ../input//snmulti_QTL_sample_participant_lookup_re  \
    --cwd ../output/eQTL/  \
    --annotation_fasta  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy_ERCC.fasta \
    --genotype_list /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.region_list.txt   \
    --region_list /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene_id.unique.gtf \
    --grmFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/GRM/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.loco_grm_list.txt \
    --queue     premium  --yml  ~/minerva.yml --run  --exe_dir /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline  -s build &

In [309]:
tail nohup.out

INFO: Step phenotype_partition_by_chrom (index=77) is ignored with signature constructed
INFO: Step phenotype_partition_by_chrom (index=76) is ignored with signature constructed
INFO: Step phenotype_partition_by_chrom (index=78) is ignored with signature constructed
INFO: Step phenotype_partition_by_chrom (index=79) is ignored with signature constructed
INFO: Step phenotype_partition_by_chrom (index=80) is ignored with signature constructed
INFO: Step phenotype_partition_by_chrom (index=81) is ignored with signature constructed
INFO: Step phenotype_partition_by_chrom (index=82) is ignored with signature constructed
INFO: Step phenotype_partition_by_chrom (index=83) is ignored with signature constructed
INFO: phenotype_partition_by_chrom output:   ../output/eQTL/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.phenotype_by_chrom_gct_files.txt ../output/eQTL/data_preprocessing/Astro.EC/phen

In [310]:
head /hpc/users/sunh14/snmulti_QTL/input/snmulti_QTL_analysis_recipe.tsv

Theme	molecular_pheno	genotype_file	covariate_file
Astro.EC	/hpc/users/sunh14/snmulti_QTL/output/molecular_phenotype/gene_exp_dreamlet/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.gz	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.bed	/hpc/users/sunh14/snmulti_QTL/input/snmulti_QTL_sample_covariates_age_msex_pmi.tsv
Astro.PFC	/hpc/users/sunh14/snmulti_QTL/output/molecular_phenotype/gene_exp_dreamlet/pb4dreamlet_subclass.Astro.PFC.low_expression_filtered.normalized.log2cpm.gct.gz	/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.bed	/hpc/users/sunh14/snmulti_QTL/input/snmulti_QTL_sample_covariates_age_msex_pmi.tsv
Astro.STG	/hpc/users/sunh14/snmulti_QTL/output/molecular_phenotype/gene_exp_dreamlet/pb4dreamlet_subclass.Astro.STG.low_expression_filtered.normalized.log2cpm.gct.gz	/sc/arion/pro

In [295]:
cat /hpc/users/sunh14/.sos/d60acd702f58c6c1/MMQTL_2_19684_96de9767.sh

cd /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/eQTL/association_scan/MMQTL/MMQTL
~/sunh14/git/MMQTL/MMQTL26 -P /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/eQTL/association_scan/MMQTL/MMQTL/ENSG00000160213/phenotype_input_tmp -Z /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/eQTL/association_scan/MMQTL/MMQTL/ENSG00000160213/genotype_input_tmp -a /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene_id.unique.gtf -T ENSG00000160213  -o snmulti_QTL_analysis_recipe.tsv.ENSG00000160213. --grm_file /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/eQTL/association_scan/MMQTL/MMQTL/ENSG00000160213/snmulti_QTL_analysis_recipe.tsv.ENSG00000160213._peak_1_statistical_signal.grm




In [296]:
cat /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/eQTL/association_scan/MMQTL/MMQTL/ENSG00000160213/genotype_input_tmp

/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.21.bed
/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.21.bed
/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.21.bed
/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.21.bed
/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.21.bed
/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.21.bed
/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.21.bed
/sc/arion/projects/CommonMi

In [268]:
cat ../output/caQTL/data_preprocessing/Astro.PFC/resid_phenotype/pb4dreamlet_subclass_ATAC.Astro.PFC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.gz.step_resid_factor_exp.stderr

INFO: Running peason_residual: 
INFO: t9d01bef3f50bd3a1 submitted to premium with job id 108624335
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
ERROR: [peason_residual]: [t9d01bef3f50bd3a1]: No result is received for task t9d01bef3f50bd3a1


In [266]:
cat ../output/caQTL/data_preprocessing/Oligo.STG/resid_phenotype/pb4dreamlet_subclass_ATAC.Oligo.STG.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.gz.step_resid_factor_exp.stderr

INFO: Running peason_residual: 
INFO: tb2abdf700dbfbba1 submitted to premium with job id 108628899
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
INFO: Waiting for the completion of 1 task.
ERROR: [peason_residual]: [tb2abdf700dbfbba1]: No result is received for task tb2abdf700dbfbba1


In [214]:
cat /hpc/users/sunh14/.sos/d60acd702f58c6c1/factor_74_b7736048.sh

 sos run /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline/pipeline/covariate_hidden_factor_dreamlet.ipynb Marchenko_PC \
   --cwd ../output/caQTL/data_preprocessing/VLMC.STG/covariates \
   --phenoFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/caQTL/data_preprocessing/VLMC.STG/resid_phenotype/pb4dreamlet_subclass_ATAC.VLMC.STG.low_expression_filtered.normalized.log2cpm.gct.cov_pca.residual.bed.gz  \
   --container container/PCAtools.sif  \
   --covFile  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/caQTL/data_preprocessing/VLMC.STG/covariates/snmulti_QTL_sample_covariates_age_msex_pmi.pca.gz \
   --walltime 24h \
   --numThreads 8 \
   --iteration 1000 \
   --N 0  -J 50 -c /hpc/users/sunh14/minerva.yml -q premium



In [270]:
nohup sos run pipeline/mmQTL_analysis_commands_atac.ipynb MMQTL \
    --recipe   /hpc/users/sunh14/snmulti_QTL/input/snmulti_atac_QTL_analysis_recipe.tsv \
    --annotation_gtf /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.gtf   \
    --sample_participant_lookup ../input//snmulti_QTL_sample_participant_lookup_re  \
    --peak_annotation /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/Peak_cell_anno_region_list \
    --cwd ../output/caQTL/  \
    --annotation_fasta  /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy_ERCC.fasta \
    --genotype_list /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.region_list.txt   \
    --grmFile /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL//output/GRM/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.loco_grm_list.txt \
    --queue     premium  --yml  ~/minerva.yml --run  --exe_dir /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/xqtl-pipeline -s build > atac_analysis_script  &

[4] 28902


: 1